<a href="https://colab.research.google.com/github/margotbrady/dssystems/blob/main/bradydataproject1/bradydataproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project has been completed by Margot Brady (apn7yj)


Links to download the datasets:

spongebob_episodes dataset : https://www.kaggle.com/datasets/myticalcat/spongebob-squarepants-episodes-dataset?select=spongebob_episodes.json

exams dataset:https://www.kaggle.com/datasets/whenamancodes/students-performance-in-exams

Here are all of the libraries I used for the project:

In [71]:
#install libraries

import pandas as pd
import requests
import json
import glob
import os
import ast
!pip install SQLAlchemy
from sqlalchemy import create_engine


This is the 'extract' portion of the ETL project. It pulls the file name and path from the newest uploaded local file in colab, and then saves it into a dataframe.

In [110]:
# Function to get the newest file uploaded
def getfilepath(directory='/content/'):
    # Get a list of all files in the directory with their full paths
    files = glob.glob(directory + '*')

    # Get all the files in the directory, not subdirectories
    files = [i for i in files if os.path.isfile(i)]

    # If no files are found, return None
    if not files:
        return None

    # Find the most recently added file by using max() with modification time as the key
    filepath = max(files, key=os.path.getmtime)

    # Return the path of the newest file
    return filepath


# This is the 'extract' part, where we are fetching the data that was just uploaded

def getdata():
  filepath = getfilepath() #gets the filepath
  try:
    if filepath is None:
      print("No files found in the directory.")
      return
    print(f"Newest file uploaded: {filepath}") #confirming the file

  # Determines the file type
    if filepath.endswith('.json'):
      filetype = 'JSON'
      df = pd.read_json(filepath)
    elif filepath.endswith('.csv'):
      filetype = 'CSV'
      df = pd.read_csv(filepath)
    else:
      # In case the user uploaded a different file type.
      print("Unsupported file type. Please upload a JSON or CSV file.")
      return

    #Show number of columns and rows in original datset uploaded
    num_records = len(df)
    num_columns = len(df.columns)
    print(f"Data Summary:\n- Number of records: {num_records}\n- Number of columns: {num_columns}")
    print("Columns in the dataset:", df.columns.tolist())

  #In case there's an error with the data
  except ValueError as e:
        print(f"Error loading data: {e}")
        return None, None

  return df, filepath,filetype  # Return both the dataframe and the filepath for future use

This converts the datatype to the new requested file-type, and saves it locally.

In [112]:
#CSV converter, works for any file type because it will have been converted to a dataframe first
def convert_to_csv(df, output_path):
    df.to_csv(output_path, index=False)
    #saves the data to a local file
    print(f"Data converted to CSV and saved at {output_path}")

def convert_to_json(df, output_path):
#JSON converter, works for any file type because it will have been converted to a dataframe first
    df.to_json(output_path, orient='records', lines=True)
    print(f"Data converted to JSON and saved at {output_path}")

def convert_to_sql():
    #Convert columns with lists or dictionaries to JSON strings
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, (list, dict))).any():
            df[col] = df[col].apply(json.dumps)  # Convert lists/dicts to JSON strings
    filename = input('What is the name of your file?')
    engine = create_engine('sqlite:///' + filename + '_sql.db', echo=False)
    df.to_sql(filename, con=engine, if_exists='replace', index=False)
    print(f"\nFile converted successfully to SQL table as {filename}_converted.db!")

def transform():
  while True:
    #Asks for an output type
    output = input("What data type would you like your data converted to? Please enter either 'JSON', 'SQL', or 'CSV'. ")

    if datatype == output.upper():
      print('Please enter a datatype different from your original file.')
    elif output.upper() != datatype and output.upper() == 'CSV':
      output_path = input("Please provide the path to save the converted file (e.g., /content/output.csv or output.json): ")
      convert_to_csv(df, output_path)
      break
    elif output.upper()!= datatype and output.upper() == 'JSON':
      output_path = input("Please provide the path to save the converted file (e.g., /content/output.csv or output.json): ")
      convert_to_json(df, output_path)
      break
    elif output.upper()!=datatype and output.upper() == 'SQL':
      convert_to_sql()
      break
    else:
      print("\nError: Unsupported output format selected.")









The first example uses the JSON Spongebob Episode dataset, and converts it to CSV.

In [108]:
#Example use: JSON to CSV


print("Please upload your data source. It must be either a JSON or CSV file.")

#Fetch
df, storedfilepath,datatype = getdata()
print('\nStored filepath: ' +storedfilepath)


# The dataset loaded weirdly into the dataframe, so we need to take the information in the info column and create new columns.

def parse_info_column(info):
  if isinstance(info, str):
    return ast.literal_eval(info)
  return info

# Apply the parsing function to the 'info' column
df['info'] = df['info'].apply(parse_info_column)
''
# Expand the 'info' dictionary into separate columns
df = pd.concat([df.drop(columns=['info']), df['info'].apply(pd.Series)], axis=1)


#Show number of columns and rows in original datset uploaded
num_records = len(df)
num_columns = len(df.columns)
print(f"Data Summary:\n- Number of records: {num_records}\n- Number of columns: {num_columns}")


#transform:
transform()

Please upload your data source. It must be either a JSON or CSV file.
Newest file uploaded: /content/spongebob_episodes.json
Data Summary:
- Number of records: 583
- Number of columns: 4
Columns in the dataset: ['title', 'info', 'characters', 'url']

Stored filepath: /content/spongebob_episodes.json
Data Summary:
- Number of records: 583
- Number of columns: 26
What data type would you like your data converted to? Please enter either 'JSON', 'SQL', or 'CSV'. json
Please enter a datatype different from your original file.
What data type would you like your data converted to? Please enter either 'JSON', 'SQL', or 'CSV'. csv
Please provide the path to save the converted file (e.g., /content/output.csv or output.json): /content/spongebob_episodes.csv
Data converted to CSV and saved at /content/spongebob_episodes.csv


The second example uses the student exam result dataset in the CSV format and converts it into SQL.

In [113]:
# Example: CSV to SQL

print("Please upload your data source. It must be either a JSON or CSV file.")

#Fetch
df, storedfilepath,datatype = getdata()
print('\nStored filepath: ' +storedfilepath)

#Add a column

df['Exam_average'] =  df[['math score', 'reading score', 'writing score']].mean(axis=1)

#transform:
transform()

#Show number of columns and rows in original datset uploaded
num_records = len(df)
num_columns = len(df.columns)
print(f"Data Summary:\n- Number of records: {num_records}\n- Number of columns: {num_columns}")



Please upload your data source. It must be either a JSON or CSV file.
Newest file uploaded: /content/exams.csv
Data Summary:
- Number of records: 1000
- Number of columns: 8
Columns in the dataset: ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course', 'math score', 'reading score', 'writing score']

Stored filepath: /content/exams.csv
What data type would you like your data converted to? Please enter either 'JSON', 'SQL', or 'CSV'. sql
What is the name of your file?exams

File converted successfully to SQL table as exams_converted.db!
Data Summary:
- Number of records: 1000
- Number of columns: 9
